## Análise de dados de vendas para segmentação de clientes, recomendação de produtos e previsão de demanda.

Dados disponíveis em: https://www.kaggle.com/datasets/rohitsahoo/sales-forecasting

### 4) Funções para criar a aplicação

Importando as bibliotecas:

In [1]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
from plotly import io as pio

from statsmodels.tsa.statespace.sarimax import SARIMAX

#### Função para retorno dos gráficos de análise:

In [2]:
# Carregando o dataframe com todas vendas:
df = pd.read_csv('./../data/vendas.csv', index_col=0, 
                 parse_dates=['Order Date'], dayfirst=True).sort_values('Order Date', ascending=True)

In [3]:
# Criando arquivo com ticket médio:
df_ticket = pd.DataFrame({'segmento': df.Segment.unique(), 
                          'total_vendas': [np.round(df[df.Segment==n]['Sales'].sum(), 2) for n in df.Segment.unique()], 
                          'num_ordens': [len(df[df.Segment==n]['Order ID'].unique()) for n in df.Segment.unique()]})

df_ticket['ticket_medio'] = np.round(df_ticket.total_vendas/df_ticket.num_ordens, 2)

df_ticket.to_csv('./../data/df_ticket.csv')

In [4]:
# Gráfico com total de vendas por segmento:
def total_seg(df_ticket):
    fig = go.Figure()
    fig.add_trace(go.Bar(x=df_ticket.segmento.unique(), y=df_ticket['total_vendas'], 
                         text=[(str(round(n/10e2))+'K') for n in df_ticket['total_vendas']], 
                         textposition='auto'))

    fig.update_layout(title_text='Total de vendas por segmento', 
                      title_x=0.5, title_font_size=10, 
                      width=400, height=150, margin=dict(l=0, r=0, b=0, t=15))
    
    fig.update_yaxes(visible=False)
    
    return (pio.to_html(fig, include_plotlyjs='cdn', validate=False))

In [5]:
# Gráfico com ticket médio (total de vendas/núm de ordens de compra) para cada segmento:
def ticket_seg(df_ticket):
    fig = go.Figure()
    fig.add_trace(go.Bar(x=df_ticket.segmento, y=df_ticket['ticket_medio'], 
                         text=df_ticket['ticket_medio'], 
                         textposition='auto'))

    fig.update_layout(title_text='Ticket médio por segmento', 
                      title_x=0.5, title_font_size=10, 
                      width=400, height=150, margin=dict(l=0, r=0, b=0, t=15))
    
    fig.update_yaxes(visible=False)
    
    return (pio.to_html(fig, include_plotlyjs='cdn', validate=False))

In [6]:
# Criando arquivo com vendas por categoria:
df_cats = df.copy()
df_cats['year'] = df_cats['Order Date'].dt.year
df_cats = df_cats.groupby(['Segment', 'Category', 'year'], as_index=False).agg({'Sales': 'sum'})
df_cats.to_csv('./../data/df_cats.csv')

In [7]:
# Gráfico com total de vendas para cada categoria a depender do segmento:
def total_cat(seg, df_cats):
    df_cats = df_cats[df_cats.Segment==seg]
    
    fig = go.Figure()
    fig.add_trace(go.Bar(x=df_cats.groupby('Category', as_index=False).agg({'Sales': 'sum'})['Category'], 
                         y=df_cats.groupby('Category', as_index=False).agg({'Sales': 'sum'})['Sales'], 
                         text=[(str(round(n/10e2))+'K') for n in df_cats.groupby(
                             'Category', as_index=False).agg({'Sales': 'sum'})['Sales']], 
                         textposition='auto'))

    fig.update_layout(title_text=f'Total de vendas por categoria no segmento {seg}', 
                      title_x=0.5, title_font_size=10, 
                      width=400, height=150, margin=dict(l=0, r=0, b=0, t=15))
    
    fig.update_yaxes(visible=False)
  
    return (pio.to_html(fig, include_plotlyjs='cdn', validate=False))

In [8]:
# Gráfico com vendas ano a ano para cada categoria a depender do segmento:
def total_cat_ano(seg, df_cats):
    df_cats = df_cats[df_cats.Segment==seg]
    cats = df_cats.Category.unique()

    fig = go.Figure()
    for n in cats:
        fig.add_trace(go.Scatter(x=[str(n) for n in df_cat.year.unique()], 
                                 y=df_cat[df_cat.Category==n]['Sales'],
                                 name = n))

    fig.update_layout(title_text=f'Total de vendas por categoria no segmento Consumer', 
                      title_x=0.5, title_font_size=10, 
                      legend={'orientation':'h', 
                              'y':1, 'yanchor': 'top', 
                              'x':0, 'xanchor': 'left', 
                              'font':{'size':7}, 'itemclick':'toggle'}, 
                      width=900, height=150, margin=dict(l=0, r=0, b=0, t=15))
    
    fig.update_yaxes(visible=False)

    return (pio.to_html(fig, include_plotlyjs='cdn', validate=False))

#### Função para retorno dos produtos recomendados:

In [9]:
def indicados(segmento, subcategoria, produto):
    # Carregando os dataframes:
    associacoes_nome = segmento.lower().replace(' ', '_') + '_associacoes.csv'
    prop_vendas_nome = segmento.lower().replace(' ', '_') + '_prop_vendas.csv'
    
    associacoes = pd.read_csv(f'./../data/{associacoes_nome}', index_col=0)
    prop_vendas = pd.read_csv(f'./../data/{prop_vendas_nome}', index_col=0)
    
    # Buscando os mais vendidos e os recomendados:
    vendidos = prop_vendas[prop_vendas['Sub-Category'] == subcategoria][
        prop_vendas['Product Name'] != produto]['Product Name'].to_list()

    recomendados = associacoes[associacoes.antecedents == produto]['consequents'].to_list()
    
    # Criando o dataframe e evitando erros para produtos sem recomendações:
    diff = len(vendidos) - len(recomendados)
    if diff > 0:
        [recomendados.append('Sem mais recomendações') for n in range(diff)]
    elif diff < 0:
        [vendidos.append('Sem mais recomendações') for n in range(-diff)]
    else:
        pass
    
    df = pd.DataFrame({'Quem comprou esse produto também comprou': recomendados, 
                   'Você também pode gostar de': vendidos})
    
    return df.head(5)

#### Função para previsão de vendas para mais 12 meses com base na sub-categoria:

In [10]:
# Carregar o dataframe de prev_vendas antes de utilizar a função:
df_prev = pd.read_csv('./../data/df_prev.csv', parse_dates=['Order Date'], index_col=0)

# Função para criar o gráfico com previsões de vendas:
def prev_vendas(df_prev, seg, subcategoria):
    # Definindo o dataframe com as vendas:
    df_prev = df_prev[(df_prev.Segment==seg) & (df_prev['Sub-Category']==subcategoria)]
    
    # Definindo range de datas:
    idx = pd.date_range(df_prev.index[0], periods=60, freq='M')
    idx = pd.to_datetime(idx.map(lambda x: x.strftime('%Y-%m-01')))
    
    # Preenchendo os meses sem vendas com zero:
    for n in idx[:48]:
        if n in df_prev.index:
            pass
        else:
            df_prev = pd.concat([df_prev, pd.DataFrame({'Segment': df_prev.Segment.iloc[0], 
                                                        'Sub-Category': df_prev['Sub-Category'].iloc[0], 
                                                        'Sales': 0}, index=[n])])
    df_prev.sort_index(inplace=True)
    
    # Criando o modelo e realizando as previsões:
    model = SARIMAX(endog=df_prev['Sales'], order=(0,1,1), seasonal_order=(0,1,1,12), trend='c', 
                enforce_invertibility=False, enforce_stationarity=False)

    sarimax = model.fit(method='nm')
    
    # Realizando as previsões e criando o dataframe:
    preds = pd.DataFrame([n for n in np.round(sarimax.predict(start=1, end=60))], 
                         index=idx, columns=['Valores'])
    
    preds['Valores'].iloc[:48] = df_prev['Sales'].to_list()[:48]
    
    # Criando o gráfico:
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=preds.index[:48], y=preds['Valores'].iloc[:48],
                                 name = 'Observados'))

    fig.add_trace(go.Scatter(x=preds.index[47:], y=preds['Valores'].iloc[47:],
                                 name = 'Previstos'))

    fig.update_layout(title_text=f'Previsão de vendas para a subcategoria {subcategoria} no segmento {seg}', 
                      title_font_color='black', title_x=0.5, title_font_size=10, 
                      legend={'orientation':'h', 
                              'y':1, 'yanchor': 'top', 
                              'x':0, 'xanchor': 'left', 
                              'font':{'size':8, 'color':'black'}, 'itemclick':'toggle'}, 
                      margin=dict(l=0, r=0, b=0, t=15))
    
    fig.update_xaxes(tickfont_size=9, tickfont_color='black')
    fig.update_yaxes(visible=False)
    fig.show()
    return (pio.to_html(fig, include_plotlyjs='cdn', validate=False))

In [11]:
def lista_subcat(seg):
    prop_vendas_nome = seg.lower().replace(' ', '_') + '_prop_vendas.csv'
    prop_vendas = pd.read_csv(f'./../data/{prop_vendas_nome}', index_col=0)
    nomes = [n for n in prop_vendas['Sub-Category'].unique()]
    return np.sort(nomes)

def lista_produtos(seg, subcategoria):
    prop_vendas_nome = seg.lower().replace(' ', '_') + '_prop_vendas.csv'
    prop_vendas = pd.read_csv(f'./../data/{prop_vendas_nome}', index_col=0)
    nomes = prop_vendas[prop_vendas['Sub-Category']==subcategoria]['Product Name'].to_list()
    return np.sort(nomes)

### Fim.